In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
import numpy as np

# Cargar modelo
detect_fn = tf.saved_model.load("saved_models/640_model/saved_model")

In [2]:
from datasets import load_dataset

raw_dataset = load_dataset("ilsilfverskiold/traffic-camera-norway-images")


c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Cargar imagen de prueba
image = raw_dataset['train'][12]['image']

# Preprocesar imagen
input_tensor = tf.convert_to_tensor(image)
input_tensor = input_tensor[tf.newaxis, ...]

# Ejecutar inferencia
detections = detect_fn(input_tensor)

In [ ]:
drawn_image = image.copy()

detections = {key:value for key,value in detections.items()}
print(detections["detection_classes"][0])

# Filter out boxes with classes and confidence > 0.5
index_list = [3, 4, 6, 8] # (cars, pickups, motorcycles, trucks)

mask = np.isin(detections["detection_classes"][0], index_list)
boxes = detections["detection_boxes"][0][mask]
scores = detections["detection_scores"][0][mask]

# Filter boxes with confidence > threshold
threshold = 0.3
boxes = boxes[scores > threshold]
scores = scores[scores > threshold]

for box in boxes:
    ymin, xmin, ymax, xmax = [int(val * (image.height if i % 2 == 0 else image.width)) for i, val in enumerate(box)]
    print(xmin, ymin, xmax, ymax)

    drawn_image = np.array(drawn_image)
    drawn_image[ymin:ymax, xmin:xmin+2] = [255, 0, 0]
    drawn_image[ymin:ymax, xmax:xmax+2] = [255, 0, 0]
    drawn_image[ymin:ymin+2, xmin:xmax] = [255, 0, 0]
    drawn_image[ymax:ymax+2, xmin:xmax] = [255, 0, 0]
    drawn_image = Image.fromarray(drawn_image)

drawn_image.show()

[ 3.  3.  6.  6.  8.  3.  8.  8.  3.  8.  6.  3.  3.  8.  3.  8.  3.  8.
  8.  3.  8.  7.  8. 17.  3.  3.  3.  6.  6.  3.  6.  3.  3.  3.  7.  3.
  3.  3.  8.  8.  3.  3.  3. 15.  3.  3.  3.  3.  3.  8.  6.  3. 19.  3.
  3. 15. 28.  3.  3.  3.  6.  3.  7.  3.  3.  3.  3.  3.  7. 15.  3.  6.
  3.  3.  3.  8.  3.  3.  7.  8.  3.  8.  3.  3.  3.  3.  1.  3.  3.  1.
  3.  8.  3.  3.  3.  3.  8.  3.  3.  3.]
195 378 327 471
36 494 352 574
429 309 477 365
195 378 327 471
783 226 844 267
480 273 511 302
786 244 836 269
429 309 477 365
428 284 459 308


In [ ]:
detected_classes = detections['detection_classes'][0].numpy() #guarda las clases detectadas
deteceted_scores = detections['detection_scores'][0].numpy() #guarda la puntuación  de cada predicción, para considerarla le puse que sea mayor a .50

count = 0

for i in range(len(detected_classes)):
  if detected_classes[i] in index_list and deteceted_scores[i] >= .50: #podría ser mayor a .50 para mayor precisión
    count += 1

print(f'hay {count} vehículos en la imagen.')

hay 3 vehículos en la imagen.
